In [2]:
import pandas
import numpy as np

from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential,load_model, clone_model

2022-01-13 14:15:33.768138: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2022-01-13 14:15:34.434601: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libnvinfer.so.7
2022-01-13 14:15:34.435719: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libnvinfer_plugin.so.7


# build a linear interpolator from cst screws position to cmp

In [3]:
# load the sensitivity analysis file
df_map = pandas.read_csv('./data/refined_cmp.csv')

In [5]:
df_map.head(10)

,Screw Name,Screw Position,f1,f2,f3,f4,f5,f6,f7,f8,...,BW2_3,BW3_4,BW4_5,BW5_6,BW6_7,BW7_8,BW1_4,BW5_8,BW2_4,BW5_7
0,f1,-0.122628,4459.331877,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-34.137853,-20.991896,5.519425,-51.974149
1,f1,0.044039,4528.221945,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-35.795280,-20.991896,5.519425,-51.974149
2,f1,0.210706,4599.309671,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-37.723693,-20.991896,5.519425,-51.974149
3,f1,0.377372,4667.789921,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-39.256968,-20.991896,5.519425,-51.974149
4,f1,0.544039,4733.792593,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-40.760893,-20.991896,5.519425,-51.974149
5,f1,0.710706,4799.186552,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-42.248214,-20.991896,5.519425,-51.974149
6,f1,0.877372,4860.583741,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-43.091299,-20.991896,5.519425,-51.974149
7,f1,1.044039,4916.933775,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-44.815407,-20.991896,5.519425,-51.974149
8,f1,1.210706,4966.910709,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-46.048862,-20.991896,5.519425,-51.974149
9,f1,1.377372,5013.892344,4909.805051,4910.187242,4905.361985,4904.906347,4855.855686,4914.368104,4807.948762,...,160.15168,113.120324,118.093813,102.780765,126.691115,168.846577,-46.784577,-20.991896,5.519425,-51.974149


In [ ]:
# the linear interpolator from Reza
def map_screws_to_cmp(self):        
    pred_cmp = filter8p4z_cst_calib.cmp_tuned
    # Jay:use_finer_f_only is set to false by default
    if self.use_finer_f_only:
        for i in range(self.numScrews):
            tmp_cmp = np.zeros((self.numScrews, 1))
            for j in range(self.numScrews):
                if i < filter8p4z_cst_calib.nrof_res:
                    tmp_cmp[j] = np.interp(self.screws_curr_pos[i], self.screws_map_pos_finer_f_only[i, :], self.cmp_map_finer_f_only[j, i * filter8p4z_cst_calib.nrof_map_pts_finer_f_only : (i + 1) * filter8p4z_cst_calib.nrof_map_pts_finer_f_only])
                else:
                    tmp_cmp[j] = np.interp(self.screws_curr_pos[i], self.screws_map_pos[i, :], self.cmp_map[j, i * filter8p4z_cst_calib.nrof_map_pts : (i + 1) * filter8p4z_cst_calib.nrof_map_pts])

            pred_cmp = pred_cmp + tmp_cmp - filter8p4z_cst_calib.cmp_tuned
    else:
        for i in range(self.numScrews):
            tmp_cmp = np.zeros((self.numScrews, 1))
            for j in range(self.numScrews):
                tmp_cmp[j] = np.interp(self.screws_curr_pos[i], self.screws_map_pos[i, :], self.cmp_map[j, i * filter8p4z_cst_calib.nrof_map_pts : (i + 1) * filter8p4z_cst_calib.nrof_map_pts])

            pred_cmp = pred_cmp + tmp_cmp - filter8p4z_cst_calib.cmp_tuned
        
    return pred_cmp


In [4]:
# modified version
f_tuned = np.array([4766.4, 4909.8, 4910.2, 4905.4, 4904.9, 4855.8, 4914.4, 4807.9]) # order = f1 .. f8
coup_tuned = np.array([194.28, 160.15, 113.12, 118.09, 102.78, 126.69, 168.85, -41.57, -20.99, 5.52, -51.97])# order = BW12, 23, ..., 78, 14, 58, 24, 57
    
nrof_map_pts = 10
nrof_tunable_par = 19
    
screws_map_pos = np.zeros((nrof_tunable_par, nrof_map_pts))
cmp_map = np.zeros((nrof_tunable_par, nrof_map_pts * nrof_tunable_par))

# Loading the mapping from the csv file for future interpolations
df_map = pandas.read_csv('./data/refined_cmp.csv')


def map_screws_to_cmp_modified(screws_curr_pos):
    pred_cmp = np.concatenate((f_tuned, coup_tuned))
    pred_cmp = pred_cmp.reshape((19, 1))
    
    cmp_tuned = np.concatenate((f_tuned, coup_tuned))
    cmp_tuned = cmp_tuned.reshape((19, 1))

    for i in range(nrof_tunable_par):
        screws_map_pos[i, :] = np.array(df_map['Screw Position'][i * nrof_map_pts : (i + 1) * nrof_map_pts])
    for i in range(nrof_map_pts * nrof_tunable_par):
        for j in range(nrof_tunable_par):
            cmp_map[j, i] = np.array(df_map.iloc[i, j + 2])

    for i in range(19):
        tmp_cmp = np.zeros((19, 1))
        for j in range(19):
            tmp_cmp[j] = np.interp(screws_curr_pos[i], screws_map_pos[i, :], 
                                   cmp_map[j, i * nrof_map_pts : (i + 1) * nrof_map_pts])

        pred_cmp = pred_cmp + tmp_cmp - cmp_tuned
    
    return pred_cmp


## load data

In [5]:
screw_pos_1300 = np.loadtxt('data/screw_position_CST_matrix_random_1300.csv',delimiter=',')
cmp_1300 = np.loadtxt('data/cmp_matrix_19_seed1300.csv',delimiter=',')

action_CST_1300 = np.loadtxt('data/action_in_CST_matrix_rand_seed1300.csv',delimiter=',')
action_filttune_full_1300 = np.loadtxt('data/action_in_filttune_matrix_19_seed1300.csv',delimiter=',')

golden_CST = np.array([0.627372, 0.849978, 0.950813, 0.875983, 0.866188, 1.03133, 0.671715, 0.085633, 1.69804,
                       0.508077, 0.803913, 1.08792, -0.844056, -0.484051, 1.71896, 0, 0, 0, 0]);
golden_cmp = cmp_1300[:,-1]

##### NOTE that golden_cmp is slightly different from Reza's data

In [6]:
golden_cmp

array([4766.29661844, 4909.78532573, 4910.2190568 , 4905.37944946,
       4904.92446961, 4855.83295124, 4914.39764996, 4807.78726567,
        194.2611782 ,  160.14845903,  113.10592641,  118.10442858,
        102.79119083,  126.7522391 ,  169.01408253,  -41.58845174,
        -21.02683131,    5.54469257,  -52.02574186])

In [31]:
cmp_1300[:,0].shape

(19,)

In [37]:
tmp = map_screws_to_cmp_modified(screw_pos_1300[:,0])
print(tmp)
tmp.reshape(19,)

[[ 4.72147742e+03]
 [ 4.94545966e+03]
 [ 4.85879171e+03]
 [ 4.89647984e+03]
 [ 4.90489332e+03]
 [ 4.84682887e+03]
 [ 4.87989926e+03]
 [ 4.83851168e+03]
 [ 2.05579630e+02]
 [ 1.50376547e+02]
 [ 1.34360300e+02]
 [ 1.05952012e+02]
 [ 1.18400414e+02]
 [ 1.38159048e+02]
 [ 1.67214006e+02]
 [-3.92318815e+01]
 [-2.34296935e+01]
 [ 2.72179182e+00]
 [-4.97572356e+01]]


array([ 4.72147742e+03,  4.94545966e+03,  4.85879171e+03,  4.89647984e+03,
        4.90489332e+03,  4.84682887e+03,  4.87989926e+03,  4.83851168e+03,
        2.05579630e+02,  1.50376547e+02,  1.34360300e+02,  1.05952012e+02,
        1.18400414e+02,  1.38159048e+02,  1.67214006e+02, -3.92318815e+01,
       -2.34296935e+01,  2.72179182e+00, -4.97572356e+01])

In [21]:
action_filttune_full_1300[:,0] + golden_cmp - cmp_1300[:,0]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -3.55271368e-15,  4.44089210e-16, -7.10542736e-15])

In [22]:
screw_pos_1300.shape

(19, 4802)

## Screw_pos = action_CST + golden_CST, cmp = action_filttune + golden_cmp

### predicted cmp using linear interpolator

In [41]:
cmp_pred_lininter = np.zeros([19,4802])

for i in range(4802):
    cmp_pred_lininter[:,i] = map_screws_to_cmp_modified(screw_pos_1300[:,i]).reshape(19,)

In [42]:
cmp_pred_lininter[:,0:15]

array([[ 4.72147742e+03,  4.76894867e+03,  4.80605552e+03,
         4.78835101e+03,  4.78877592e+03,  4.67836130e+03,
         4.69371367e+03,  4.80650724e+03,  4.71008538e+03,
         4.78927195e+03,  4.80705665e+03,  4.77257279e+03,
         4.83253060e+03,  4.83828410e+03,  4.83984360e+03],
       [ 4.94545966e+03,  4.88367578e+03,  4.86468185e+03,
         4.94486863e+03,  4.91346708e+03,  4.89606294e+03,
         4.90311170e+03,  4.89294504e+03,  4.91984337e+03,
         4.88511654e+03,  4.90148057e+03,  4.86523321e+03,
         4.91794688e+03,  4.83889334e+03,  4.94750253e+03],
       [ 4.85879171e+03,  4.95546868e+03,  4.93110057e+03,
         4.93428820e+03,  4.96216462e+03,  4.93653870e+03,
         4.96972367e+03,  4.84624575e+03,  4.93270085e+03,
         4.92089030e+03,  4.86503256e+03,  4.96566787e+03,
         4.90909807e+03,  4.90361400e+03,  4.87192916e+03],
       [ 4.89647984e+03,  4.93093739e+03,  4.92918162e+03,
         4.92631249e+03,  4.86785062e+03,  4.95872697

### predicted cmp using regression model

In [49]:
# load the trained model
reg_CF = load_model('model/model_comp_small_ST13_0_CF.h5')

2022-01-11 12:48:39.627535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 12:48:39.628198: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5650d63444f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-01-11 12:48:39.628225: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2022-01-11 12:48:39.628502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 12:48:39.629076: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySiz

In [53]:
action_filttune_pred = reg_CF.predict(action_CST_1300.transpose())

In [57]:
cmp_pred_reg = np.zeros([19,4802])
tmp = action_filttune_pred.transpose()

for i in range(4802):
    cmp_pred_reg[:,i] = tmp[:,i] + golden_cmp

### metrics

In [59]:
mse_lininter = metrics.mean_squared_error(cmp_1300[:,0:4802], cmp_pred_lininter)
mse_reg = metrics.mean_squared_error(cmp_1300[:,0:4802], cmp_pred_reg)

print("The full metrics")
print("--------------------------------------")
print('MSE of interpolator is {}'.format(mse_lininter))
print('MSE of regression is {}'.format(mse_reg))

The full metrics
--------------------------------------
MSE of interpolator is 19.581417011309377
MSE of regression is 7.478222035527945


In [60]:
mse_lininter_F = metrics.mean_squared_error(cmp_1300[0:8,0:4802], cmp_pred_lininter[0:8,:])
mse_reg_F = metrics.mean_squared_error(cmp_1300[0:8,0:4802], cmp_pred_reg[0:8,:])

print("The metrics of F-type screws")
print("--------------------------------------")
print('MSE of interpolator is {}'.format(mse_lininter_F))
print('MSE of regression is {}'.format(mse_reg_F))

The metrics of F-type screws
--------------------------------------
MSE of interpolator is 28.130352340914
MSE of regression is 9.06283477314004


# Modify regression model to map cst screws position to cmp